<a href="https://colab.research.google.com/github/rhqtmfajfl/python-study/blob/master/RNN/05_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM의 `return_sequences`, `return_state` 이해하기
* `return_sequences` : 매 타임 스텝의 출력 여부 결정 결정
* `return_state` : 제일 마지막 스테이트 출력 여부 결정

In [10]:
import numpy as np
from tensorflow.keras.layers import LSTM

In [11]:
sample_train = np.random.randn(1, 4, 5) # N L I N 입력차원의 개 : (데이터 개수, 최대타임스탭, 임베딩차원)

return_sequences =False, return_state =False 확인
* return _

In [12]:
# 제일 마지막 HIDDEN STATE 만 반환 된다.

last_hidden_state = LSTM(3, return_sequences=False, return_state=False)(sample_train)
print(last_hidden_state) # 마지막 히든 스테이트만 나오게 된다.

tf.Tensor([[0.143588   0.18108416 0.10203055]], shape=(1, 3), dtype=float32)


`return_sequences=False`, `return_state=True`

* `hidden_states` = `last_hidden_state`
* `last_cell_state`

In [13]:
hidden_states, last_hidden_state, last_cell_state = LSTM(3, return_sequences=False, return_state=True)(sample_train)
print("hidden_states : {}".format(hidden_states))
print("last_hidden_state : {}".format(last_hidden_state))
print("last_cell_state : {}".format(last_cell_state))

hidden_states : [[-0.10380044  0.08630534  0.30688265]]
last_hidden_state : [[-0.10380044  0.08630534  0.30688265]]
last_cell_state : [[-0.2548225   0.14245956  0.74640447]]


`return_sequences=True`, `return_state=False`


In [14]:
hidden_states = LSTM(3, return_sequences=True, return_state=False)(sample_train)
print("hidden_states : {} / shape : {}".format(hidden_states, hidden_states.shape))

hidden_states : [[[-0.00298128  0.09771238 -0.01691602]
  [-0.01515853  0.02933614 -0.05290803]
  [ 0.0254513   0.10560504 -0.11466618]
  [-0.01330078 -0.0996694   0.13423991]]] / shape : (1, 4, 3)


`return_sequences=True`,`return_state=True`

In [15]:
hidden_states, last_hidden_state, last_cell_state = LSTM(3, return_sequences=True, return_state=True)(sample_train)
print("hidden_states : {}".format(hidden_states))
print("last_hidden_state : {}".format(last_hidden_state))
print("last_cell_state : {}".format(last_cell_state))

hidden_states : [[[-0.03499834  0.10745714 -0.10676076]
  [ 0.00822138 -0.01179479 -0.16099004]
  [ 0.34729257 -0.06188013  0.01959288]
  [ 0.20290439  0.01924237 -0.01521631]]]
last_hidden_state : [[ 0.20290439  0.01924237 -0.01521631]]
last_cell_state : [[ 0.4077325   0.03281772 -0.02206334]]


## Seq2Seq 챗봇 만들기


In [16]:
!pip install konlpy

In [17]:
import random # 나중에 데이터 셔플링 할 예정
import tensorflow as tf
from konlpy.tag import Okt

# 하이퍼 파라미터

In [18]:
num_epochs = 200
vocab_size = 2000

# Encoder

In [19]:
class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    self.emb = tf.keras.layers.Embedding(vocab_size, 64) # 임베딩차원도 64개
    
    # 제일 마지막 state를 리턴해야 context vector가 나옴!
    self.lstm = tf.keras.layers.LSTM(512, return_sequences=False, return_state=True)


  def call(self, training=False):
    # 데이터가 들어온다 
    # 임베딩
    x = self.emb(x)

    # Encoder에서는 context vector 만 얻어내면 되기 때문에 각 time 별 state는필요가 없다.
    _, h, c = self.lstm(x)

    #context vector return
    return h, c

# Decoder

In [23]:
class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    # 여기도 임베딩이 있어야 하낟.
    self.emb= tf.keras.layers.Embedding(vocab_size, 64) # 언어 인코딩 만들려면 각 언어들마다의 임베딩 벡터를 만들어야 한다.
    self.lstm = tf.keras.layers.LSTM(512, return_sequences = True, return_state=True )
    
    #각 셀마다 2000개의 output을내고, 어떤 단어가추정 되었을 지를 계산
    self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax') # 단어 2000개의 데이터에서 softmax의 결과를 낸다.

  def call(self, inputs, training=False):
    #입력된  데이터들
    x, h, c = inputs # shifted, (hidden_state, cell_state) #한칸 밀린데이터가 들어온다.
    # 히든 스테이트고 셀스테이트기도 하지만 그냥 값이다.
    x = self.emb(x) # 입력한 단어에 대한 임베딩벡터
     # wx+b에 대한 값만 기억하고 있기


    #y_ : 해당 시퀀스의 hidden_state
    y_, h, c = self.lstm(x, initial_state=[h,c]) # initial_state : 초기화할 hidden_state, cell_state를 지정
    
    # y 값이랑 h, c 가 다음 단계로 넘어갈 값이 된다.

    y = self.dense(y_)

    return y, h, c

# Seq2Seq

In [24]:
class Seq2seq(tf.keras.Model):
  #sos최초값을 넣느다.eos도 넣는다.
  def __init__(self, sos, eos):
    self.sos = sos # decoder에서 사용되어질 sos
    self.eos = eos # encoder에서 사용되어질 eos
    
    self.enc = Encoder()
    self.dec = Decoder() # enc와 dec를 엮어 줄 것이다.

  def call(self, inputs, training=False):
    if training: # 훈련시에서는 Teacher Forcing 때문에 정답이 들어옴
      x, y  = inputs # 정답이 그대로들어감 기본 입력데이ㅓ(output_labels, shifted labels)

      h, c = self.enc(x) # context vector가 등장 (들어감) 인코더에 대한 컨텍스트 벡터를 받아옴

      # y의 shape은 전체 데이터를 모두 가지고 있는 상태가 된다.
      # y.shape :(N, 65, 64) # 최대 단어수 64개가 64개씩 들어간다.
      y, _, ___ = self.dec((y,h, c))#teacher forcing Decoder의 입력으로 Shifted Output을 넣어줌

      #길이가 정해진 값들이 들어오게 됨
      

      return y # 훈련된 결과가 나옴
    else: # 테스트 할 때는 x 만 들어온다.
      x = inputs
      # x엗 해나 히든과 셀스테이트
      h, c = self.enc(x) # last_cell_state, last_hidden_state

      # 시퀀셜 스테이트는없다.

      #<sos> 입력
      #1. <sos> 토큰을 tensor 배열화 시켜야함
      y = tf.convert_to_tensor(self.sos) # 0 rank tensor로 변환
      y = tf.reshape(y, (1,1)) # <sos> 가 (1,1) 형식으로 변환 (1배치, 1타임 스텝)을 의미, embedding 레이어에 넣을 예정

      # 최대 입력 길이 만큼의 공간을미리 만들어 놓자
      seq = tf.TensorArray(tf.int32, 64) # 64개의 텐서 배열 만들어 놓기
    
      # tensorflow의 session 환경에서 for 무을 조금더 빠르게 돌릴 수 있따.
      for idx in tf.range(64):
        # 제일 첨음엔 <sos>, 인코더의 h, c (context vector)가 들어간다.

        ###########################################################################################################여기가 핵심
        y, h, c = self.dec([y,h,c]) # 여기서 리턴 받는 y는 softmax의 결과 다음 시퀀스에 대한 소프트맥스 결과  
        #######################################################################################################
        #
        
        y = tf.cas(tf.argmax(y, axis= -1), dtype=tf.int32) #-1은 y는 0부터 1999중에 가장 큰것이 등장

        # 한문장 들어가서 하나의 결과, 1배치 들어가니까 나오는 것도 1개
        y = tf.reshape(y, (1,1)) # (1 배치, 1단어를 의미하기 위해 reshape - 테스트 할 때는 배치를 1로 설정할 예정) # 한문장

        seq = seq.write(idx, y) # 순서대로 write

        if y == self.eos:
          break

      return tf.reshape(seq.stack(), (1,64))